In [1]:
#!pip install chromadb openai tiktoken transformers datasets langchain

In [2]:
#!pip show chromadb

In [3]:
#!pip install wget unzip

In [4]:
import requests

url = "https://www.dropbox.com/scl/fi/p7d0zjnrefjne94j3ec2b/new_articles.zip?rlkey=izy5z44eofgx5fmjn8tif4y99&dl=1"
response = requests.get(url)

with open("new_articles.zip", "wb") as f:
    f.write(response.content)

print("Download complete.")


Download complete.


In [5]:
!unzip -q new_articles.zip -d new_articles

replace new_articles/05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

OSError: [Errno 5] Input/output error

In [ ]:
pwd

'/Users/apple/Desktop/DEV/vectordb'

In [6]:
import requests
import os

url = "https://api.openai.com/v1/models"

headers = {
    "Authorization": os.getenv('OPEN_AI_APIKEY')
    #'Authorization': OPEN_AI_APIKEY
}

response = requests.get(url, headers=headers)
print(response.status_code)
print(response.json())


401
{'error': {'message': 'Missing bearer authentication in header', 'type': 'invalid_request_error', 'param': None, 'code': None}}


In [7]:
print("API key:", os.getenv('OPEN_AI_APIKEY'))


API key: None


In [8]:
!echo $OPEN_AI_APIKEY
!echo $openai_key_1

In [9]:
from openai import OpenAI

# initialize the client
#client = OpenAI(api_key=os.getenv('openai_key_1'))
client = OpenAI(api_key='')

# make a request
response = client.chat.completions.create(
    model="gpt-4o-mini",  # or any available model from your /models response
    messages=[
        {"role": "system", "content": "You are a poet."},
        {"role": "user", "content": "Write a haiku about AI."}
    ]
)

# print output
print(response.choices[0].message.content)


AuthenticationError: Error code: 401 - {'error': {'message': "You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
response = client.responses.create(
  model="gpt-4o-mini",
  input="write a brief message about ASI, rise of super intelligent machines, and the future of humanity",
  store=True,
)

print(response.output_text)

As we stand on the brink of Artificial Superintelligence (ASI), we are entering a new era where machines may surpass human intelligence in nearly every aspect. This remarkable potential holds both profound promise and significant challenges for humanity. ASI could lead to breakthroughs in medicine, climate change solutions, and more, enhancing our quality of life. However, it also raises critical ethical questions about control, safety, and the socio-economic implications of such advancements. 

It's essential that we prioritize responsible development and governance of superintelligent systems to ensure they align with human values and goals. The future of humanity will depend on our ability to navigate this transformative technology wisely, ensuring that AI serves as a partner in our evolution rather than a threat to our existence. Let us approach this frontier with caution, creativity, and a commitment to a better world for all.


Import Libraries

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

Load text files

In [ ]:
loader=DirectoryLoader("new_articles",
                       glob="./*.txt",
                       #glob="05-03-ai-replace-tv-writers-strike.txt",
                       loader_cls=TextLoader)
document=loader.load()

In [ ]:
#document

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#split into smaller chunks for semantic search
text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text= text_splitter.split_documents(document)

In [ ]:
len(text)

233

In [ ]:
#Creating DB object

In [ ]:
# import os
# from langchain import embeddings

# persist_directory= 'db'
# embedding= OpenAIEmbeddings(
#   model="text-embedding-3-small",
#   openai_api_key="")

# vectordb= Chroma.from_documents(documents=text,
#                                 embedding=embedding,
#                                 persist_directory=persist_directory)



In [10]:
#Embed those chunks using all-MiniLM-L6-v2 (a lightweight Hugging Face model).
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import os

persist_directory = 'db'

#create the embedding
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

#embeddings= embedding.embed_documents([doc.page_content for doc in text])

# create the vectordb object
vectordb = Chroma.from_documents(
    documents=text,
    embedding=embedding,
    persist_directory=persist_directory
)

# Query the vector store
query = "AI replacing TV writers"
results = vectordb.similarity_search(query, k=2)

# Display results
for i, doc in enumerate(results, 1):
    print(f"\nResult {i}:\n")
    print(doc.page_content[:500])
    print("-" * 80)

/var/folders/pb/q49x3lbx6313whr2jx93rk180000gn/T/ipykernel_10036/454964295.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/Users/apple/pinokio/api/facefusion-pinokio.git/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'text' is not defined

In [ ]:
#!pip install sentence-transformers

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})


In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ''


In [13]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token="")
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="auto")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7
)

llm = HuggingFacePipeline(pipeline=pipe)

prompt = "Explain the impact of AI on creative writing."
response = llm.invoke(prompt)

print(response)



/Users/apple/pinokio/api/facefusion-pinokio.git/.env/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]/Users/apple/pinokio/api/facefusion-pinokio.git/.env/lib/python3.10/site-packages/huggingface_hub/file_download.py:801: UserWarning: Not enough free disk space to download the file. The expected file size is: 4899.04 MB. The target location /Users/apple/.cache/huggingface/hub/models--mistralai--Mixtral-8x7B-Instruct-v0.1/blobs only has 3802.28 MB free disk space.
  warnings.warn(
/Users/apple/pinokio/api/facefusion-pinokio.git/.env/lib/python3.10/site-packages/huggingface_hub/file_download.py:801: UserWarning: Not enough free disk space to download the file. The expected file size is: 4983.00 MB. The targ

KeyboardInterrupt: 

In [11]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)
query = "Explain the impact of AI on creative writing."
result = qa_chain.run(query)

ModuleNotFoundError: No module named 'langchain_core.pydantic_v1'

In [12]:
# 1️⃣ Imports
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.schema import Document
import os

# -------------------------------------------------------------------
# 2️⃣ Embedding setup (for document storage and retrieval)
persist_directory = 'db'

embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Example documents (replace with your real scraped Bible or text chunks)
text = [
    Document(page_content="AI is transforming creative industries, including writing and music."),
    Document(page_content="TV writers are exploring how AI can assist with story development."),
    Document(page_content="Many creatives fear AI might replace some jobs but open new opportunities.")
]

# Create or load the Chroma vector database
vectordb = Chroma.from_documents(
    documents=text,
    embedding=embedding,
    persist_directory=persist_directory
)
# vectordb.persist()  # optional if you want to save it permanently

# -------------------------------------------------------------------
# 3️⃣ Load the LLM (Mixtral 8x7B Instruct)
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=""
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7
)

llm = HuggingFacePipeline(pipeline=pipe)

# -------------------------------------------------------------------
# 4️⃣ RAG Query + Context Assembly
query = "AI replacing TV writers"
retrieved_docs = vectordb.similarity_search(query, k=2)

context = "\n\n".join([doc.page_content for doc in retrieved_docs])

prompt = f"""
You are an expert writer and analyst.

Use the context below to answer the question clearly and insightfully.

Context:
{context}

Question:
{query}

Answer:
"""

# -------------------------------------------------------------------
# 5️⃣ Generate final answer from LLM
response = llm.invoke(prompt)
print("\n🧠 RAG Response:\n")
print(response)


ModuleNotFoundError: No module named 'langchain_core.memory'